In [6]:
!pip install ultralytics

Извлекаем метки из файлов

In [7]:
import pandas as pd  # для работы с таблицами и датасетами
import os  # для работы с осями

train_dir = 'images/train/'
train_filepaths = [os.path.join(train_dir, f) for f in os.listdir(train_dir)]   # создаем список путей изображений

labels = []
for path in train_filepaths:
    # Извлекаем метку из имени файла
    label = os.path.basename(path).split('_')[1].split('.')[0]
    labels.append(label)

train_df = pd.DataFrame({'filepath': train_filepaths, 'label': labels})  # датафрейм путей файлов и меток

# Посмотрим, что получилось и сколько у нас классов
print(train_df.head())
print(train_df['label'].value_counts())

                        filepath   label
0    images/train/0695_bluzy.jpg   bluzy
1   images/train/9524_bryuki.jpg  bryuki
2    images/train/7283_bluzy.jpg   bluzy
3  images/train/11326_bryuki.jpg  bryuki
4    images/train/7760_bluzy.jpg   bluzy
label
bluzy     7074
bryuki    6402
Name: count, dtype: int64


Прописываем трансформации

In [8]:
from torchvision import transforms

# Трансформации для тренировочного набора (с аугментацией)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # Случайное отражение
    transforms.RandomRotation(10),     # Случайный поворот
    transforms.ToTensor(),             # Преобразование в тензор
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Нормализация
])

# Трансформации для тестового набора (без аугментации!)
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Разбиваем датасет на тренировочный и тестовый

In [9]:
from sklearn.model_selection import train_test_split

# Разделяем наш DataFrame на две части: 80% на обучение, 20% на валидацию
train_subset_df, val_subset_df = train_test_split(
    train_df,
    test_size=0.2,       # Доля данных для валидации
    random_state=42,     # Для воспроизводимости результатов
    stratify=train_df['label'] # Очень важно! Сохраняет баланс классов
)

Класс для создания датасета тензоров

In [10]:
from torch.utils.data import Dataset
from PIL import Image
import torch

class ClothingDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Конструктор класса.
        :param dataframe: pandas DataFrame с колонками 'filepath' и 'label'.
        :param transform: конвейер трансформаций (наш transforms.Compose).
        """
        self.df = dataframe
        self.transform = transform
        # Создаем словарь для преобразования текстовых меток в числа
        self.label_map = {'bluzy': 0, 'bryuki': 1}

    def __len__(self):
        """Возвращает общее количество объектов в датасете."""
        return len(self.df)

    def __getitem__(self, idx):
        """
        Возвращает ОДИН объект (картинку и метку) по его индексу.
        Это самая важная часть!
        """
        # 1. Получаем путь к файлу и метку из DataFrame
        image_path = self.df.iloc[idx]['filepath']
        label_str = self.df.iloc[idx]['label']

        # 2. Открываем изображение с помощью библиотеки PIL
        image = Image.open(image_path).convert("RGB")

        # 3. Применяем наш конвейер трансформаций
        if self.transform:
            image_tensor = self.transform(image)

        # 4. Преобразуем текстовую метку в число
        label_int = self.label_map[label_str]
        # Преобразуем число в тензор PyTorch
        label_tensor = torch.tensor(label_int, dtype=torch.long)

        return image_tensor, label_tensor

Создаем датасеты и даталоадер

In [11]:
from torch.utils.data import DataLoader

# Создаем экземпляр Dataset для обучающей выборки, передаем ему наш DataFrame и train_transforms
train_dataset = ClothingDataset(train_subset_df, transform=train_transforms)

# Создаем экземпляр Dataset для валидационной выборки, передаем ему val_transforms (без аугментации!)
val_dataset = ClothingDataset(val_subset_df, transform=test_transforms) # Используем test_transforms

# Определяем размер батча
BATCH_SIZE = 32

# Создаем DataLoader'ы
# shuffle=True для обучающего загрузчика - очень важно! Перемешивает данные каждую эпоху.
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False) # Для валидации перемешивать не нужно

Загружаем предобученную модель YOLO11

In [12]:
from ultralytics import YOLO

model = YOLO('yolo11m-cls.pt')

# Проверим модель
print(model.info())

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/sergei/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLO11m-cls summary: 106 layers, 11,634,216 parameters, 0 gradients, 40.6 GFLOPs
(106, 11634216, 0, 40.6263808)


Готовим данные в структуре, понятной YOLO

In [13]:
import shutil
import os

def prepare_yolo_folders(df, base_dir):
    for idx, row in df.iterrows():
        label = row['label']
        src = row['filepath']
        dst_dir = os.path.join(base_dir, label)
        os.makedirs(dst_dir, exist_ok=True)
        shutil.copy(src, dst_dir)

# Создаём структуру папок
os.makedirs('lamoda_dataset/train', exist_ok=True)
os.makedirs('lamoda_dataset/val', exist_ok=True)

prepare_yolo_folders(train_subset_df, 'lamoda_dataset/train')
prepare_yolo_folders(val_subset_df, 'lamoda_dataset/val')


Запускаем обучение YOLO11

In [16]:
pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
# Обучение
import torch

model.train(
    data='lamoda_dataset',  # Папка с train и val
    epochs=10,
    imgsz=224,  # Можно оставить 224 для скорости
    batch=32,
    device='mps'
)

# После обучения модель будет сохранена в runs/classify/trainX/

Ultralytics 8.3.191 🚀 Python-3.13.5 torch-2.8.0 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=lamoda_dataset, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, proj

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x422460e60>
curves: []
curves_results: []
fitness: 0.9987017810344696
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9974035620689392, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9987017810344696}
save_dir: PosixPath('runs/classify/train2')
speed: {'preprocess': 0.04486422770212229, 'inference': 0.309693093102407, 'loss': 4.1713799074988605e-06, 'postprocess': 1.6937684457754655e-05}
task: 'classify'
top1: 0.9974035620689392
top5: 1.0

In [25]:
# Папка с тестовыми изображениями
test_dir = 'images/test/'

# Запускаем предсказания
results = model.predict(source=test_dir, imgsz=224, device='mps', verbose=False)

# Собираем результаты
file_names = []
predictions = []

for r in results:
    file_names.append(os.path.basename(r.path))        # имя файла
    cls_id = int(r.probs.top1)                         # индекс класса
    predictions.append(r.names[cls_id])                # имя класса (bluzy или bryuki)

# Проверяем длины списков
print(f"Количество файлов: {len(file_names)}, количество предсказаний: {len(predictions)}")

# Создаем DataFrame
submission = pd.DataFrame({'index': file_names, 'label': predictions})

# Сортируем по id, чтобы порядок совпадал с Kaggle
submission = submission.sort_values('index')

# Сохраняем
submission.to_csv('submission.csv', index=False)
print("Файл submission.csv готов!")

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Количество файлов: 3369, количество предсказаний: 3369
Файл submission.csv готов!
